# Evaluating a Splade-v2 mrobust-pt pre-trained model

# imports/configs

In [1]:
!nvidia-smi

Thu Jun 29 04:39:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
data_dir = '/content/gdrive/MyDrive/unicamp/IA368DD/final-project'

In [4]:
# install splade deps from forked repo
!pip install git+https://github.com/naver/splade.git -q
# fix version problem
!pip install hydra-core --upgrade -q
# missing dep
!pip install pytrec_eval -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 64.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [5]:
import torch
print(torch.__version__)

2.0.1+cu118


In [6]:
# clone forked splade repo
!git clone https://github.com/tcvieira/splade.git splade-mine

Cloning into 'splade-mine'...
remote: Enumerating objects: 565, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 565 (delta 196), reused 166 (delta 148), pack-reused 297
Receiving objects: 100% (565/565), 3.08 MiB | 6.85 MiB/s, done.
Resolving deltas: 100% (284/284), done.


In [23]:
!cd splade-mine && git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), 568 bytes | 568.00 KiB/s, done.
From https://github.com/tcvieira/splade
   ef6d1de..075c119  main       -> origin/main
Updating ef6d1de..075c119
Fast-forward
 conf/retrieve_evaluate/mmarco-pt.yaml  | 2 +-
 conf/retrieve_evaluate/mrobust-pt.yaml | 2 +-
 2 files changed, 2 insertions(+), 2 deletions(-)


In [8]:
# clone forked splade repo
#!git clone https://github.com/naver/splade.git splade

In [9]:
#!cd splade && git checkout hf && git status

# dataset

In [10]:
!cp gdrive/MyDrive/unicamp/IA368DD/final-project/mrobust-eval-dataset.zip splade-mine/data

In [11]:
!unzip splade-mine/data/mrobust-eval-dataset.zip -d splade-mine/data

Archive:  splade-mine/data/mrobust-eval-dataset.zip
   creating: splade-mine/data/mrobust-pt/
   creating: splade-mine/data/mrobust-pt/qrel/
   creating: splade-mine/data/mrobust-pt/dev_queries/
   creating: splade-mine/data/mrobust-pt/full_collection/
  inflating: splade-mine/data/mrobust-pt/qrel/qrel.tsv  
  inflating: splade-mine/data/mrobust-pt/qrel/qrel.json  
  inflating: splade-mine/data/mrobust-pt/qrel/qrels.robust04.txt  
  inflating: splade-mine/data/__MACOSX/mrobust-pt/qrel/._qrels.robust04.txt  
  inflating: splade-mine/data/mrobust-pt/dev_queries/raw.tsv  
  inflating: splade-mine/data/__MACOSX/mrobust-pt/dev_queries/._raw.tsv  
  inflating: splade-mine/data/mrobust-pt/full_collection/raw.tsv  
  inflating: splade-mine/data/mrobust-pt/full_collection/raw-alpha-ids.tsv  
  inflating: splade-mine/data/__MACOSX/mrobust-pt/full_collection/._raw-alpha-ids.tsv  


In [12]:
!rm -rf splade-mine/data/__MACOSX

# indexing mrobust pt

In [13]:
!mkdir -p experiments/splade_mmarco_pt_4M/

In [14]:
!cp -R /content/gdrive/MyDrive/unicamp/IA368DD/final-project/experiments/splade_mmarco_pt_4M/* experiments/splade_mmarco_pt_4M/

In [15]:
!cd splade-mine && SPLADE_CONFIG_NAME=config_splade_mrobust-pt python3 -m splade.index \
  config.checkpoint_dir=/content/experiments/splade_mmarco_pt_4M/checkpoint \
  config.index_dir=/content/experiments/splade_mrobust_pt/index \
  config.out_dir=/content/experiments/splade_mrobust_pt/out

/content/splade-mine/splade/index.py:12: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=CONFIG_PATH, config_name=CONFIG_NAME)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_splade_mrobust-pt': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
config:
  lr: 2.0e-05
  seed: 123
  gradient_accumulation_steps: 1
  weight_decay: 0.01
  validation_metrics:
  - MRR@10
  - recall@100
  - recall@200
  - recall@500
 

In [16]:
!cp -R /content/experiments/splade_mrobust_pt/index/*  /content/gdrive/MyDrive/unicamp/IA368DD/final-project/experiments/splade_mrobust_pt/index/

cp: target '/content/gdrive/MyDrive/unicamp/IA368DD/final-project/experiments/splade_mrobust_pt/index/' is not a directory


In [17]:
!zip -r index_mrobust_4M.zip experiments/splade_mrobust_pt/index/

  adding: experiments/splade_mrobust_pt/index/ (stored 0%)
  adding: experiments/splade_mrobust_pt/index/index_stats.json (stored 0%)
  adding: experiments/splade_mrobust_pt/index/doc_ids.pkl (deflated 66%)
  adding: experiments/splade_mrobust_pt/index/array_index.h5py (deflated 36%)
  adding: experiments/splade_mrobust_pt/index/index_dist.json (deflated 64%)


In [20]:
!cp index_mrobust_4M.zip /content/gdrive/MyDrive/unicamp/IA368DD/final-project/experiments/splade_mrobust_pt/

In [ ]:
from google.colab import runtime
runtime.unassign()

# retrieve and evaluate mrobust pt

In [22]:
import pytrec_eval
pytrec_eval.supported_measures

{'11pt_avg',
 'G',
 'P',
 'Rndcg',
 'Rprec',
 'Rprec_mult',
 'binG',
 'bpref',
 'gm_bpref',
 'gm_map',
 'infAP',
 'iprec_at_recall',
 'map',
 'map_cut',
 'ndcg',
 'ndcg_cut',
 'ndcg_rel',
 'num_nonrel_judged_ret',
 'num_q',
 'num_rel',
 'num_rel_ret',
 'num_ret',
 'recall',
 'recip_rank',
 'relative_P',
 'relstring',
 'runid',
 'set_F',
 'set_P',
 'set_map',
 'set_recall',
 'set_relative_P',
 'success',
 'utility'}

In [24]:
!cd splade-mine && SPLADE_CONFIG_NAME=config_splade_mrobust-pt python3 -m splade.retrieve \
  config.checkpoint_dir=/content/experiments/splade_mmarco_pt_4M/checkpoint \
  config.index_dir=/content/experiments/splade_mrobust_pt/index \
  config.out_dir=/content/experiments/splade_mrobust_pt/out

/content/splade-mine/splade/evaluate.py:12: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=CONFIG_PATH, config_name=CONFIG_NAME)
/content/splade-mine/splade/retrieve.py:13: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=CONFIG_PATH, config_name=CONFIG_NAME)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_splade_mrobust-pt': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/d